In [ ]:
import json
import pickle
from pathlib import Path

In [ ]:
def insert_instruction_actions(time_list, phetio_id):
    '''Insert time when the student opened/closed instruction during the simulation. phetioID stands for the name of the action'''
    
    for time_to_insert in time_list:
        inserted = False

        for index, log_entry in enumerate(parsed_json['logs']):
            if log_entry is None:
                continue
            if 'time' not in log_entry:
                continue
            if log_entry['time'] > time_to_insert:
                parsed_json['logs'].insert(index, {'time': time_to_insert, 'phetioID': phetio_id})
                inserted = True
                break

        if not inserted:
            parsed_json['logs'].append({'time': time_to_insert, 'phetioID': phetio_id})


# PARSER

In [ ]:
def bll_parser(parsed_json):
    
    logs_dict = []
    log = {}
    count_action = 0
    table_point = 1
    
    restore_width = []
    restore_solution = []
    restore_wave = []
    restore_ruler = []
    restore_conc = []
    

    row = []
    list_df = []

    for i in range(182, len(parsed_json['logs'])):

        if parsed_json['logs'][i] == None:
            pass   
        
        # Restore sim from points
        elif 'labBook.restoreButton' in parsed_json['logs'][i]['phetioID']:
            log['index'] = count_action
            log['action'] = 'restore_sim'
            log['timestamp'] = parsed_json['logs'][i]['time']

            try:
                int(parsed_json['logs'][i]['phetioID'][-2:])
            except ValueError:
                log['table_point'] = int(parsed_json['logs'][i]['phetioID'][-1])
            else:
                log['table_point'] = int(parsed_json['logs'][i]['phetioID'][-2:])

            for k in range(len(parsed_json['logs'][i+1]['children'])):
                if parsed_json['logs'][i+1]['children'][k]['phetioID'] == 'beersLawLab.beersLawScreen.model.cuvette.widthProperty':
                    restore_width.append(parsed_json['logs'][i+1]['children'][k]['time']) 
                elif parsed_json['logs'][i+1]['children'][k]['phetioID'] == "beersLawLab.beersLawScreen.model.solutionProperty":
                    restore_solution.append(parsed_json['logs'][i+1]['children'][k]['time']) 
                elif parsed_json['logs'][i+1]['children'][k]['phetioID'] == "beersLawLab.beersLawScreen.model.light.wavelengthProperty":
                    restore_wave.append(parsed_json['logs'][i+1]['children'][k]['time']) 
                elif parsed_json['logs'][i+1]['children'][k]['phetioID'] == "beersLawLab.beersLawScreen.model.ruler.positionProperty":
                    restore_ruler.append(parsed_json['logs'][i+1]['children'][k]['time'])
                elif 'concentrationProperty' in parsed_json['logs'][i+1]['children'][k]['phetioID'] and 'rangeProperty' not in parsed_json['logs'][i+1]['children'][k]['phetioID']:
                    restore_conc.append(parsed_json['logs'][i+1]['children'][k]['time'])
                    if 'children' in parsed_json['logs'][i+1]['children'][k]:
                        if parsed_json['logs'][i+1]['children'][k]['children'][0]['phetioID'] == 'beersLawLab.beersLawScreen.model.solutionInCuvette.concentrationProperty':
                            restore_conc.append(parsed_json['logs'][i+1]['children'][k]['children'][0]['time'])
                else:
                    pass
            
            logs_dict.append(log)
            log = {}
            count_action += 1

        # Watching instruction
        elif parsed_json['logs'][i]['phetioID'] == 'open_instruction':
            log['index'] = count_action
            log['action'] = 'open_instruction'
            log['timestamp'] = parsed_json['logs'][i]['time']

            logs_dict.append(log)
            log = {}
            count_action += 1

        elif parsed_json['logs'][i]['phetioID'] == 'close_instruction':
            log['index'] = count_action
            log['action'] = 'close_instruction'
            log['timestamp'] = parsed_json['logs'][i]['time']

            logs_dict.append(log)
            log = {}
            count_action += 1 
            
        # Clicks / Touches
        elif parsed_json['logs'][i]['phetioID'] == 'beersLawLab.general.controller.input.mouseDownAction' or parsed_json['logs'][i]['phetioID'] == "beersLawLab.general.controller.input.touchStartAction":
            log['index'] = count_action
            log['action'] = 'click_start'
            log['timestamp'] = parsed_json['logs'][i]['time']

            logs_dict.append(log)
            log = {}
            count_action += 1

        elif parsed_json['logs'][i]['phetioID'] == 'beersLawLab.general.controller.input.mouseUpAction' or parsed_json['logs'][i]['phetioID'] == "beersLawLab.general.controller.input.touchEndAction":
            log['index'] = count_action
            log['action'] = 'click_end'
            log['timestamp'] = parsed_json['logs'][i]['time']

            logs_dict.append(log)
            log = {}
            count_action += 1

        elif parsed_json['logs'][i]['phetioID'] == 'beersLawLab.general.controller.input.clickAction':
            log['index'] = count_action
            log['action'] = 'click_sum'
            log['timestamp'] = parsed_json['logs'][i]['time']

            logs_dict.append(log)
            log = {}
            count_action += 1

        # Width
        elif parsed_json['logs'][i]['phetioID'] == 'beersLawLab.beersLawScreen.model.cuvette.widthProperty' and parsed_json['logs'][i]['time'] not in restore_width:
            log['index'] = count_action
            log['action'] = 'width'
            log['timestamp'] = parsed_json['logs'][i]['time']
            log['old_value'] = parsed_json['logs'][i]['data']['oldValue']
            log['new_value'] = parsed_json['logs'][i]['data']['newValue']

            if len(parsed_json['logs'][i]['children']) > 1 and parsed_json['logs'][i+4]['phetioID'] == 'beersLawLab.beersLawScreen.model.detector.absorbanceProperty':
                log['old_abs'] = round(parsed_json['logs'][i+4]['data']['oldValue'], 2)
                log['new_abs'] = round(parsed_json['logs'][i+4]['data']['newValue'], 2)
            elif len(parsed_json['logs'][i]['children']) == 1 and 'children' not in parsed_json['logs'][i]['children'][0]:
                pass
            elif len(parsed_json['logs'][i]['children']) == 1 and parsed_json['logs'][i+2]['phetioID'] == 'beersLawLab.beersLawScreen.model.detector.absorbanceProperty':
                log['old_abs'] = round(parsed_json['logs'][i+2]['data']['oldValue'], 2)
                log['new_abs'] = round(parsed_json['logs'][i+2]['data']['newValue'], 2)
            else:
                print(i)
                print('width issue')
                break

            logs_dict.append(log)
            log = {}
            count_action += 1

        # Wavelength
        elif parsed_json['logs'][i]['phetioID'] == 'beersLawLab.beersLawScreen.model.light.wavelengthProperty' and parsed_json['logs'][i-1]['phetioID'] != 'beersLawLab.beersLawScreen.model.solutionProperty' and parsed_json['logs'][i]['time'] not in restore_wave:
            log['index'] = count_action
            log['action'] = 'wavelength'
            log['timestamp'] = parsed_json['logs'][i]['time']
            log['old_value'] = parsed_json['logs'][i]['data']['oldValue']
            log['new_value'] = parsed_json['logs'][i]['data']['newValue']

            if 'children' in parsed_json['logs'][i]:
                if parsed_json['logs'][i+1]['phetioID'] == "beersLawLab.beersLawScreen.model.solutionInCuvette.molarAbsorptivityProperty" and parsed_json['logs'][i+5]['phetioID'] == 'beersLawLab.beersLawScreen.model.detector.absorbanceProperty':
                    log['old_abs'] = round(parsed_json['logs'][i+5]['data']['oldValue'], 2)
                    log['new_abs'] = round(parsed_json['logs'][i+5]['data']['newValue'], 2)
                elif len(parsed_json['logs'][i]['children']) > 2 and parsed_json['logs'][i+1]['phetioID'] == "beersLawLab.beersLawScreen.view.wavelengthPanel.numberDisplay.valueText.stringProperty":
                    pass
                elif len(parsed_json['logs'][i]['children']) > 2 and parsed_json['logs'][i+1]['phetioID'] == "beersLawLab.beersLawScreen.model.solutionInCuvette.molarAbsorptivityProperty" and 'children' not in parsed_json['logs'][i]['children'][0]:
                    pass
                elif len(parsed_json['logs'][i]['children']) == 2 and parsed_json['logs'][i+1]['phetioID'] == "beersLawLab.beersLawScreen.view.wavelengthPanel.numberDisplay.valueText.stringProperty":
                    pass
                elif len(parsed_json['logs'][i]['children']) == 1 and parsed_json['logs'][i+1]['phetioID'] == "beersLawLab.beersLawScreen.view.wavelengthPanel.wavelengthNumberControl.decrementButton.enabledProperty":
                    pass
                else:
                    print(i)
                    print('wavelength issue')
                    break

            logs_dict.append(log)
            log = {}
            count_action += 1

         # Concentration
        elif ('drinkMix' in parsed_json['logs'][i]['phetioID'] or 'cobaltChloride' in parsed_json['logs'][i]['phetioID'] or 'cobaltIINitrate' in parsed_json['logs'][i]['phetioID'] or 'copperSulfate' in parsed_json['logs'][i]['phetioID'] or 'nickelIIChloride' in parsed_json['logs'][i]['phetioID'] or 'potassiumChromate' in parsed_json['logs'][i]['phetioID'] or 'potassiumDichromate' in parsed_json['logs'][i]['phetioID'] or 'potassiumPermanganate' in parsed_json['logs'][i]['phetioID']) and 'rangeProperty' not in parsed_json['logs'][i]['phetioID'] and parsed_json['logs'][i]['time'] not in restore_conc:
            log['index'] = count_action
            log['action'] = 'concentration'
            log['timestamp'] = parsed_json['logs'][i]['time']
            log['old_value'] = parsed_json['logs'][i]['data']['oldValue']
            log['new_value'] = parsed_json['logs'][i]['data']['newValue']

            if 'children' not in parsed_json['logs'][i]:
                pass
            elif parsed_json['logs'][i+5]['phetioID'] == 'beersLawLab.beersLawScreen.model.detector.absorbanceProperty':
                log['old_abs'] = round(parsed_json['logs'][i+5]['data']['oldValue'], 2)
                log['new_abs'] = round(parsed_json['logs'][i+5]['data']['newValue'], 2)
            else:
                print(i)
                print('concentration issue')
                break

            logs_dict.append(log)
            log = {}
            count_action += 1

        # Solution
        elif parsed_json['logs'][i]['phetioID'] == 'beersLawLab.beersLawScreen.model.solutionProperty' and parsed_json['logs'][i]['time'] not in restore_solution:
            log['index'] = count_action
            log['action'] = 'solution'
            log['timestamp'] = parsed_json['logs'][i]['time']
            
            if parsed_json['logs'][i]['data']['oldValue']['phetioID'] == "beersLawLab.beersLawScreen.model.solutions.drinkMix":
                log['old_value'] = 'drink Mix (red)'
            elif parsed_json['logs'][i]['data']['oldValue']['phetioID'] == "beersLawLab.beersLawScreen.model.solutions.cobaltChloride":
                log['old_value'] = 'Cobalt chloride (red)'
            elif parsed_json['logs'][i]['data']['oldValue']['phetioID'] == "beersLawLab.beersLawScreen.model.solutions.cobaltIINitrate":
                log['old_value'] = 'Cobalt nitrate (red)'
            elif parsed_json['logs'][i]['data']['oldValue']['phetioID'] == "beersLawLab.beersLawScreen.model.solutions.copperSulfate":
                log['old_value'] = 'Copper sulfate (blue)'
            elif parsed_json['logs'][i]['data']['oldValue']['phetioID'] == "beersLawLab.beersLawScreen.model.solutions.nickelIIChloride":
                log['old_value'] = 'Nickel chloride (green)'
            elif parsed_json['logs'][i]['data']['oldValue']['phetioID'] == "beersLawLab.beersLawScreen.model.solutions.potassiumChromate":
                log['old_value'] = 'Potassium chromate (yellow)'
            elif parsed_json['logs'][i]['data']['oldValue']['phetioID'] == "beersLawLab.beersLawScreen.model.solutions.potassiumDichromate":
                log['old_value'] = 'Potassium dichromate (orange)'
            else:
                log['old_value'] = 'Potassium permanganate (purple)'

            if parsed_json['logs'][i]['data']['newValue']['phetioID'] == "beersLawLab.beersLawScreen.model.solutions.drinkMix":
                log['new_value'] = 'drink Mix (red)'
            elif parsed_json['logs'][i]['data']['newValue']['phetioID'] == "beersLawLab.beersLawScreen.model.solutions.cobaltChloride":
                log['new_value'] = 'Cobalt chloride (red)'
            elif parsed_json['logs'][i]['data']['newValue']['phetioID'] == "beersLawLab.beersLawScreen.model.solutions.cobaltIINitrate":
                log['new_value'] = 'Cobalt nitrate (red)'
            elif parsed_json['logs'][i]['data']['newValue']['phetioID'] == "beersLawLab.beersLawScreen.model.solutions.copperSulfate":
                log['new_value'] = 'Copper sulfate (blue)'
            elif parsed_json['logs'][i]['data']['newValue']['phetioID'] == "beersLawLab.beersLawScreen.model.solutions.nickelIIChloride":
                log['new_value'] = 'Nickel chloride (green)'
            elif parsed_json['logs'][i]['data']['newValue']['phetioID'] == "beersLawLab.beersLawScreen.model.solutions.potassiumChromate":
                log['new_value'] = 'Potassium chromate (yellow)'
            elif parsed_json['logs'][i]['data']['newValue']['phetioID'] == "beersLawLab.beersLawScreen.model.solutions.potassiumDichromate":
                log['new_value'] = 'Potassium dichromate (orange)'
            else:
                log['new_value'] = 'Potassium permanganate (purple)'

            if parsed_json['logs'][i+1]['phetioID'] == "beersLawLab.beersLawScreen.model.solutionInCuvette.molarAbsorptivityProperty" and 'children' not in parsed_json['logs'][i]['children'][0]:
                pass
            elif parsed_json['logs'][i+1]['phetioID'] == "beersLawLab.beersLawScreen.model.solutionInCuvette.molarAbsorptivityProperty" and parsed_json['logs'][i+5]['phetioID'] == 'beersLawLab.beersLawScreen.model.detector.absorbanceProperty':
                log['old_abs'] = round(parsed_json['logs'][i+5]['data']['oldValue'], 2)
                log['new_abs'] = round(parsed_json['logs'][i+5]['data']['newValue'], 2)
            elif parsed_json['logs'][i+1]['phetioID'] == "beersLawLab.beersLawScreen.model.solutionInCuvette.concentrationProperty" and parsed_json['logs'][i+5]['phetioID'] == 'beersLawLab.beersLawScreen.model.detector.absorbanceProperty':
                log['old_conc'] = parsed_json['logs'][i+1]['data']['oldValue']
                log['new_conc'] = parsed_json['logs'][i+1]['data']['newValue']
                log['old_abs'] = round(parsed_json['logs'][i+5]['data']['oldValue'], 2)
                
                if parsed_json['logs'][i+7]['phetioID'] == "beersLawLab.beersLawScreen.model.solutionInCuvette.molarAbsorptivityProperty" and 'children' not in parsed_json['logs'][i+7]:
                    log['new_abs'] = round(parsed_json['logs'][i+5]['data']['newValue'], 2)
                elif parsed_json['logs'][i+7]['phetioID'] == "beersLawLab.beersLawScreen.model.solutionInCuvette.molarAbsorptivityProperty" and parsed_json['logs'][i+11]['phetioID'] == 'beersLawLab.beersLawScreen.model.detector.absorbanceProperty':
                    log['new_abs'] = round(parsed_json['logs'][i+11]['data']['newValue'], 2)
                else:
                    print(i)
                    print('solution with conc: no old_abs found')
                    break
            elif parsed_json['logs'][i+1]['phetioID'] == "beersLawLab.beersLawScreen.model.solutionInCuvette.concentrationProperty" and parsed_json['logs'][i+11]['phetioID'] == 'beersLawLab.beersLawScreen.model.detector.absorbanceProperty':
                log['old_conc'] = parsed_json['logs'][i+1]['data']['oldValue']
                log['new_conc'] = parsed_json['logs'][i+1]['data']['newValue']
                log['old_abs'] = round(parsed_json['logs'][i+11]['data']['oldValue'], 2)
                log['new_abs'] = round(parsed_json['logs'][i+11]['data']['newValue'], 2)
            
            else:
                if parsed_json['logs'][i+1]['phetioID'] == 'beersLawLab.beersLawScreen.model.light.wavelengthProperty':
                    log['old_wave'] = parsed_json['logs'][i+1]['data']['oldValue']
                    log['new_wave'] = parsed_json['logs'][i+1]['data']['newValue']
                    
                    if parsed_json['logs'][i+6]['phetioID'] == "beersLawLab.beersLawScreen.model.detector.absorbanceProperty":
                        log['old_abs'] = round(parsed_json['logs'][i+6]['data']['oldValue'], 2)
                    else:
                        if parsed_json['logs'][i+5]['phetioID'] == 'beersLawLab.beersLawScreen.model.solutionInCuvette.concentrationProperty' and parsed_json['logs'][i+9]['phetioID'] == 'beersLawLab.beersLawScreen.model.detector.absorbanceProperty':
                            log['old_abs'] = round(parsed_json['logs'][i+9]['data']['oldValue'], 2)
                        elif parsed_json['logs'][i+6]['phetioID'] == 'beersLawLab.beersLawScreen.model.solutionInCuvette.concentrationProperty' and parsed_json['logs'][i+10]['phetioID'] == 'beersLawLab.beersLawScreen.model.detector.absorbanceProperty':
                            log['old_abs'] = round(parsed_json['logs'][i+10]['data']['oldValue'], 2)
                        elif parsed_json['logs'][i+7]['phetioID'] == 'beersLawLab.beersLawScreen.model.solutionInCuvette.concentrationProperty' and parsed_json['logs'][i+11]['phetioID'] == 'beersLawLab.beersLawScreen.model.detector.absorbanceProperty':
                            log['old_abs'] = round(parsed_json['logs'][i+11]['data']['oldValue'], 2)
                        elif parsed_json['logs'][i+9]['phetioID'] == 'beersLawLab.beersLawScreen.model.solutionInCuvette.concentrationProperty' and parsed_json['logs'][i+13]['phetioID'] == 'beersLawLab.beersLawScreen.model.detector.absorbanceProperty':
                            log['old_abs'] = round(parsed_json['logs'][i+13]['data']['oldValue'], 2)
                        elif parsed_json['logs'][i+10]['phetioID'] == 'beersLawLab.beersLawScreen.model.solutionInCuvette.concentrationProperty' and parsed_json['logs'][i+14]['phetioID'] == 'beersLawLab.beersLawScreen.model.detector.absorbanceProperty':
                            log['old_abs'] = round(parsed_json['logs'][i+14]['data']['oldValue'], 2)
                        elif parsed_json['logs'][i+11]['phetioID'] == 'beersLawLab.beersLawScreen.model.solutionInCuvette.concentrationProperty' and parsed_json['logs'][i+15]['phetioID'] == 'beersLawLab.beersLawScreen.model.detector.absorbanceProperty':
                            log['old_abs'] = round(parsed_json['logs'][i+15]['data']['oldValue'], 2)
                        elif parsed_json['logs'][i+12]['phetioID'] == 'beersLawLab.beersLawScreen.model.solutionInCuvette.concentrationProperty' and parsed_json['logs'][i+16]['phetioID'] == 'beersLawLab.beersLawScreen.model.detector.absorbanceProperty':
                            log['old_abs'] = round(parsed_json['logs'][i+16]['data']['oldValue'], 2)
                        elif len(parsed_json['logs'][i]['children']) == 1 and parsed_json['logs'][i+2]['phetioID'] == "beersLawLab.beersLawScreen.model.solutionInCuvette.molarAbsorptivityProperty" and 'children' not in parsed_json['logs'][i+2]:
                            pass
                        else:
                            print(i)
                            print('solution: no old_abs found')
                            break
                else:
                    print(i)
                    print('solution wave issue')
                    break

                if len(parsed_json['logs'][i]['children']) >= 2:
                    if len(parsed_json['logs'][i]['children']) == 2 and parsed_json['logs'][i+5]['phetioID'] == 'beersLawLab.beersLawScreen.model.solutionInCuvette.concentrationProperty' and parsed_json['logs'][i+9]['phetioID'] == 'beersLawLab.beersLawScreen.model.detector.absorbanceProperty':
                        log['new_abs'] = round(parsed_json['logs'][i+9]['data']['newValue'], 2)
                        log['old_conc'] = parsed_json['logs'][i+5]['data']['oldValue']
                        log['new_conc'] = parsed_json['logs'][i+5]['data']['newValue']
                    elif len(parsed_json['logs'][i]['children']) == 2 and parsed_json['logs'][i+7]['phetioID'] == 'beersLawLab.beersLawScreen.model.solutionInCuvette.concentrationProperty' and parsed_json['logs'][i+11]['phetioID'] == 'beersLawLab.beersLawScreen.model.detector.absorbanceProperty':
                        log['new_abs'] = round(parsed_json['logs'][i+11]['data']['newValue'], 2)
                        log['old_conc'] = parsed_json['logs'][i+7]['data']['oldValue']
                        log['new_conc'] = parsed_json['logs'][i+7]['data']['newValue']
                    elif len(parsed_json['logs'][i]['children']) == 2 and parsed_json['logs'][i+9]['phetioID'] == 'beersLawLab.beersLawScreen.model.solutionInCuvette.concentrationProperty' and parsed_json['logs'][i+13]['phetioID'] == 'beersLawLab.beersLawScreen.model.detector.absorbanceProperty':
                        log['new_abs'] = round(parsed_json['logs'][i+13]['data']['newValue'], 2)
                        log['old_conc'] = parsed_json['logs'][i+9]['data']['oldValue']
                        log['new_conc'] = parsed_json['logs'][i+9]['data']['newValue']
                    elif len(parsed_json['logs'][i]['children']) == 2 and parsed_json['logs'][i+10]['phetioID'] == 'beersLawLab.beersLawScreen.model.solutionInCuvette.concentrationProperty' and parsed_json['logs'][i+14]['phetioID'] == 'beersLawLab.beersLawScreen.model.detector.absorbanceProperty':
                        log['new_abs'] = round(parsed_json['logs'][i+14]['data']['newValue'], 2)
                        log['old_conc'] = parsed_json['logs'][i+10]['data']['oldValue']
                        log['new_conc'] = parsed_json['logs'][i+10]['data']['newValue']
                    elif len(parsed_json['logs'][i]['children']) == 2 and parsed_json['logs'][i+11]['phetioID'] == 'beersLawLab.beersLawScreen.model.solutionInCuvette.concentrationProperty' and parsed_json['logs'][i+15]['phetioID'] == 'beersLawLab.beersLawScreen.model.detector.absorbanceProperty':
                        log['new_abs'] = round(parsed_json['logs'][i+15]['data']['newValue'], 2)
                        log['old_conc'] = parsed_json['logs'][i+11]['data']['oldValue']
                        log['new_conc'] = parsed_json['logs'][i+11]['data']['newValue']                        
                    elif len(parsed_json['logs'][i]['children'][0]['children']) == 3 and parsed_json['logs'][i+10]['phetioID'] == 'beersLawLab.beersLawScreen.model.solutionInCuvette.concentrationProperty' and parsed_json['logs'][i+14]['phetioID'] == 'beersLawLab.beersLawScreen.model.detector.absorbanceProperty':
                        log['new_abs'] = round(parsed_json['logs'][i+14]['data']['newValue'], 2)
                        log['old_conc'] = parsed_json['logs'][i+10]['data']['oldValue']
                        log['new_conc'] = parsed_json['logs'][i+10]['data']['newValue']                        
                    elif len(parsed_json['logs'][i]['children'][0]['children']) == 4 and parsed_json['logs'][i+6]['phetioID'] == 'beersLawLab.beersLawScreen.model.solutionInCuvette.concentrationProperty' and parsed_json['logs'][i+10]['phetioID'] == 'beersLawLab.beersLawScreen.model.detector.absorbanceProperty':
                        log['new_abs'] = round(parsed_json['logs'][i+10]['data']['newValue'], 2)
                        log['old_conc'] = parsed_json['logs'][i+6]['data']['oldValue']
                        log['new_conc'] = parsed_json['logs'][i+6]['data']['newValue']                        
                    elif len(parsed_json['logs'][i]['children'][0]['children']) == 5 and parsed_json['logs'][i+12]['phetioID'] == 'beersLawLab.beersLawScreen.model.solutionInCuvette.concentrationProperty' and parsed_json['logs'][i+16]['phetioID'] == 'beersLawLab.beersLawScreen.model.detector.absorbanceProperty':
                        log['new_abs'] = round(parsed_json['logs'][i+16]['data']['newValue'], 2)
                        log['old_conc'] = parsed_json['logs'][i+12]['data']['oldValue']
                        log['new_conc'] = parsed_json['logs'][i+12]['data']['newValue']                        
                    else:
                        print(i)
                        print('solution concentration or abs issue')
                        break
                else:
                    if parsed_json['logs'][i+2]['phetioID'] == "beersLawLab.beersLawScreen.model.solutionInCuvette.molarAbsorptivityProperty" and 'children' not in parsed_json['logs'][i+2]:
                        pass
                    elif parsed_json['logs'][i+6]['phetioID'] == 'beersLawLab.beersLawScreen.model.detector.absorbanceProperty':
                        log['new_abs'] = round(parsed_json['logs'][i+6]['data']['newValue'], 2)
                    else:
                        print(i)
                        print('solution abs issue')
                        break

            logs_dict.append(log)
            log = {}
            count_action += 1

        # Open solution menu
        elif parsed_json['logs'][i]['phetioID'] == "beersLawLab.beersLawScreen.view.solutionPanel.solutionComboBox.button.firedEmitter":
            log['index'] = count_action
            log['action'] = 'open_solution_menu'
            log['timestamp'] = parsed_json['logs'][i]['time']

            logs_dict.append(log)
            log = {}
            count_action += 1

        # Ruler
        elif parsed_json['logs'][i]['phetioID'] == 'beersLawLab.beersLawScreen.model.ruler.positionProperty' and parsed_json['logs'][i]['time'] not in restore_ruler:
            log['index'] = count_action
            log['action'] = 'ruler'
            log['timestamp'] = parsed_json['logs'][i]['time']
            log['old_value_x'] = parsed_json['logs'][i]['data']['oldValue']['x']
            log['old_value_y'] = parsed_json['logs'][i]['data']['oldValue']['y']
            log['new_value_x'] = parsed_json['logs'][i]['data']['newValue']['x']
            log['new_value_y'] = parsed_json['logs'][i]['data']['newValue']['y']

            logs_dict.append(log)
            log = {}
            count_action += 1

        # Record
        elif parsed_json['logs'][i]['phetioID'] == 'labBook.recordDataButton':
            log['index'] = count_action
            log['action'] = 'record'
            log['timestamp'] = parsed_json['logs'][i]['time']
            log['table_point'] = table_point

            logs_dict.append(log)
            log = {}
            count_action += 1
            table_point += 1

        # Delete points
        elif 'labBook.deleteButton' in parsed_json['logs'][i]['phetioID']:
            log['index'] = count_action
            log['action'] = 'delete_point_from_table'
            log['timestamp'] = parsed_json['logs'][i]['time']

            try:
                int(parsed_json['logs'][i]['phetioID'][-2:])
            except ValueError:
                log['table_point'] = int(parsed_json['logs'][i]['phetioID'][-1])
            else:
                log['table_point'] = int(parsed_json['logs'][i]['phetioID'][-2:])

            logs_dict.append(log)
            log = {}
            count_action += 1

        # Move points in table
        elif 'labBook.decrementButton' in parsed_json['logs'][i]['phetioID']:
            log['index'] = count_action
            log['action'] = 'move_down_point'
            log['timestamp'] = parsed_json['logs'][i]['time']

            try:
                int(parsed_json['logs'][i]['phetioID'][-2:])
            except ValueError:
                log['table_point'] = int(parsed_json['logs'][i]['phetioID'][-1])
            else:
                log['table_point'] = int(parsed_json['logs'][i]['phetioID'][-2:])

            logs_dict.append(log)
            log = {}
            count_action += 1

        elif 'labBook.incrementButton' in parsed_json['logs'][i]['phetioID']:
            log['index'] = count_action
            log['action'] = 'move_up_point'
            log['timestamp'] = parsed_json['logs'][i]['time']

            try:
                int(parsed_json['logs'][i]['phetioID'][-2:])
            except ValueError:
                log['table_point'] = int(parsed_json['logs'][i]['phetioID'][-1])
            else:
                log['table_point'] = int(parsed_json['logs'][i]['phetioID'][-2:])

            logs_dict.append(log)
            log = {}
            count_action += 1

        # Choose the axis
        elif parsed_json['logs'][i]['phetioID'] == 'labBook.xFeature':
            log['index'] = count_action
            log['action'] = 'xaxis'
            log['timestamp'] = parsed_json['logs'][i]['time']
            log['new_value'] = parsed_json['logs'][i]['data']['feature']

            logs_dict.append(log)
            log = {}
            count_action += 1

        elif parsed_json['logs'][i]['phetioID'] == 'labBook.yFeature':
            log['index'] = count_action
            log['action'] = 'yaxis'
            log['timestamp'] = parsed_json['logs'][i]['time']
            log['new_value'] = parsed_json['logs'][i]['data']['feature']

            logs_dict.append(log)
            log = {}
            count_action += 1

        # Axis scale
        elif parsed_json['logs'][i]['phetioID'] == 'labBook.xTransform':
            log['index'] = count_action
            log['action'] = 'xaxis_scale'
            log['timestamp'] = parsed_json['logs'][i]['time']
            log['new_value'] = parsed_json['logs'][i]['data']['feature']

            logs_dict.append(log)
            log = {}
            count_action += 1

        elif parsed_json['logs'][i]['phetioID'] == 'labBook.yTransform':
            log['index'] = count_action
            log['action'] = 'yaxis_scale'
            log['timestamp'] = parsed_json['logs'][i]['time']
            log['new_value'] = parsed_json['logs'][i]['data']['feature']

            logs_dict.append(log)
            log = {}
            count_action += 1

        # Plotting
        elif 'labBook.addToGraphCheckbox' in parsed_json['logs'][i]['phetioID']:
            log['index'] = count_action
            log['timestamp'] = parsed_json['logs'][i]['time']

            if parsed_json['logs'][i]['data']['checked'] == True:
                log['action'] = 'plot'
            else:
                log['action'] = 'remove_from_graph'

            try:
                int(parsed_json['logs'][i]['phetioID'][-2:])
            except ValueError:
                log['table_point'] = int(parsed_json['logs'][i]['phetioID'][-1])
            else:
                log['table_point'] = int(parsed_json['logs'][i]['phetioID'][-2:])

            logs_dict.append(log)
            log = {}
            count_action += 1

        else:
            row.append(i)
            row.append(parsed_json['logs'][i]['phetioID'])

            list_df.append(row)
            row = []
    
    last_timestamp = None
    for _, dictionary in reversed(list(enumerate(parsed_json['logs']))):
        if 'time' in dictionary:
            last_timestamp = dictionary['time']
            break

    logs_dict.append({'action': 'end', 'timestamp': last_timestamp})
    
    #print('End')
    
    sim_object = {}
    sim_object['group'] = parsed_json['group']
    sim_object['subgroup'] = parsed_json['subgroup']
    sim_object['session_code'] = parsed_json['session_code']
    sim_object['task'] = parsed_json['logs_type']
    sim_object['logs'] = logs_dict
    
    return sim_object

# Parsing all the files

In [ ]:
path_of_the_directory = '///' 
file = Path(path_of_the_directory ).glob('**/*.json')

instructions = []

for i in file:
    with open(i) as user_file:
        file_contents = user_file.read()
    platform = json.loads(file_contents)

    instruction = {'session_code': platform['session_code'], 'open': [], 'close': []}

    for i in range(len(platform['logs'])):
        found_close = False  # Flag to indicate whether a close instruction was found
        if platform['logs'][i]['page'] == 'PageSim1Task' and platform['logs'][i]['instr_open'] == True:
            instruction['open'].append(platform['logs'][i]['time'])
            
            # Search for the first subsequent log entry where 'instr_open' is False
            for j in range(i + 1, len(platform['logs'])):
                if platform['logs'][j]['page'] == 'PageSim1Task' and not platform['logs'][j]['instr_open']:
                    instruction['close'].append(platform['logs'][j]['time'])
                    found_close = True
                    break  # Exit the loop once a close instruction is found

    instructions.append(instruction)

In [ ]:
path_of_the_directory = '///'
file = Path(path_of_the_directory ).glob('**/*.json')

for i in file:
    with open(i) as user_file:
        file_contents = user_file.read()
    parsed_json = json.loads(file_contents)

    for item in instructions:
        if item['session_code'] == parsed_json['session_code']:
            insert_instruction_actions(item['open'], 'open_instruction')
            insert_instruction_actions(item['close'], 'close_instruction')
    
    print(parsed_json['session_code'])
    #bll_parser(parsed_json)
    
    sim_path = "/Users/kutsenok/Documents/Materials 1st exp/Parsed logs/BLL/{}_sim1.pkl".format(parsed_json['session_code'])
    with open(sim_path, 'wb') as fp:
        pickle.dump(bll_parser(parsed_json), fp)
        
print('Finish')